In [15]:
from ipywidgets import IntProgress


In [31]:
import torch
from transformers import BertTokenizer, AlbertModel


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of the model checkpoint at /Users/himon/resource/model_hub/albert_chinese_tiny/ were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Be

In [29]:
import torch
from transformers import BertTokenizer, AlbertModel


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


path = '/Users/himon/resource/model_hub/albert_chinese_tiny/'
tokenizer = BertTokenizer.from_pretrained(path)
model = AlbertModel.from_pretrained(path)

def _get_sentence_embeddings(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    print("Sentence embeddings shape:")
    print(sentence_embeddings.shape)
    return sentence_embeddings


Sentence embeddings:
torch.Size([2, 312])


In [ ]:
sentences = ['下载所需预训练参数', '修改 克隆后的代码中的']


In [20]:
import torch
from torch.autograd import Variable
import sys
# sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip

dir_st = 'data/skip-thoughts'
vocab = ['robots', 'are', 'very', 'cool', '<eos>', 'BiDiBu']
uniskip = UniSkip(dir_st, vocab)

input = Variable(torch.LongTensor([
    [1,2,3,4,0], # robots are very cool 0
    [6,2,3,4,5]  # bidibu are very cool <eos>
])) # <eos> token is optional
print(input.size()) # batch_size x seq_len

output_seq2vec = uniskip(input, lengths=[4,5])
print(output_seq2vec.size()) # batch_size x 2400

output_seq2seq = uniskip(input)
print(output_seq2seq.size()) # batch_size x seq_len x 2400

FileNotFoundError: [Errno 2] No such file or directory: 'data/skip-thoughts/utable.npy'

In [4]:
import numpy as np
import pandas as pd
import re,os,jieba
from itertools import chain

"""第一步：把文档划分成句子"""


# 文档所在的文件夹
c_root = '/Users/himon/Jobs/class/project-class/class1/resource/cnews/'

sentences_list = []
for file in os.listdir(c_root): 
    print('read file of: %s'%file)
    if 'news' not in file:
        continue
    fp = open(c_root+file,'r',encoding="utf8")
    for line in fp.readlines():
        if line.strip():
            # 把元素按照[。！；？]进行分隔，得到句子。
            line_split = re.split(r'[|。|！|；|？|]|，',line.strip())
            # [。！；？]这些符号也会划分出来，把它们去掉。
            line_split = [line.strip() for line in line_split if line.strip() not in ['。','！','？','；','，'] and len(line.strip())>1]
            sentences_list.append(line_split)
sentences_list = list(chain.from_iterable(sentences_list))
print("前10个句子为：\n")
print(sentences_list[:10])

read file of: .DS_Store
read file of: news_4.txt
read file of: news_1.txt
read file of: news_2.txt
read file of: news_3.txt
前10个句子为：

['无症状感染者恐引第二波疫情', '专家：不是主因', '关键看4月底', '新冠肺炎疫情暴发以来', '频繁出现的无症状感染者病例', '再次引起恐慌', '近日', '国家卫健委首度公布无症状感染者的情况', '截至3月31日24时', '31个省（自治区、直辖市）和新疆生产建设兵团报告新增无症状感染者130例']


In [6]:
stopwords = []

# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stopwords:
            word_list.append(word)   
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

all_vocab = set()
sentence_word_list = []
for sentence in sentences_list:   
    line_seg = seg_depart(sentence)
    all_vocab.update(line_seg)
    sentence_word_list.append(line_seg)
print("一共有",len(sentences_list),'个句子。\n')
print("前10个句子分词后的结果为：\n",sentence_word_list[:5])



一共有 1102 个句子。

前10个句子分词后的结果为：
 [['无症状', '感染者', '恐引', '第二', '波', '疫情'], ['专家', '不是', '主因'], ['关键', '看', '月底'], ['新冠', '肺炎', '疫情', '暴发', '以来'], ['频繁', '出现', '的', '无症状', '感染者', '病例']]


In [11]:
"""第三步：准备词向量"""
with open('/Users/himon/Jobs/class/project-class/class1/resource/local_vocab.txt', 'w') as result_file:
    with open('/Users/himon/Jobs/class/project-class/class1/resource/sgns.financial.word', encoding='utf-8') as f:
        next(f)
        for line in f:
            values = line.split()
            if values[0] not in all_vocab:
                continue
            result_file.write(line)